<a href="https://colab.research.google.com/github/purbid/legal_ai/blob/main/distilbert_ipc_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from pathlib import Path
import nltk
!nltk.download('reuters')

/bin/bash: -c: line 0: syntax error near unexpected token `'reuters''
/bin/bash: -c: line 0: `nltk.download('reuters')'


In [ ]:
!pip install transformers

In [2]:
import pandas
chunk_container = pandas.read_csv("/content/drive/MyDrive/judgements.csv")
chunk_container.head()
df = pandas.DataFrame(chunk_container, columns= ['details'])
df.head()

,details
0,[' 1. This reference before us arises o...
1,[' 1. This special leave petition has ...
2,"['CHANDRAMAULI KR. PRASAD, J.\n', 'CRIMINAL AP..."
3,['1. The Vasavi Co-op. Housing Society Ltd....
4,"[' Leave granted.\n', ' 2. ..."


In [ ]:
import nltk
from nltk.corpus import reuters
from nltk import sent_tokenize
text = ""
for file_id in reuters.fileids():
    text += reuters.raw(file_id)
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())
import re

LookupError: ignored

In [12]:
def read_squad(data_dict, split):
    # path = Path(path)
    # with open(path, 'rb') as f:
    #     squad_dict = json.load(f)
    squad_dict=data_dict 
    contexts = []
    questions = []
    answers = []
    train_contexts = []
    train_questions = []
    train_answers = []
    val_contexts = []
    val_questions = []
    val_answers = []
    
    #### for these following contexts(paras), I very smartly typed out the answers 
    #### instead of selecting a span in context, Hence the annotator had to
    #### select start_token as -1. We will skip them.  
    
    wrong_answers=[15,29,35,39]
    
    for x,group in enumerate(squad_dict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    if answer['answer_start']!=-1:
                        contexts.append(context)
                        questions.append(question)
                        answers.append(answer)
    
    ##### splitting wrt number split given#####
    train_end=int(len(contexts)*split)
    ## 0 to 294 
    
    for x in range(train_end):
        train_contexts.append(contexts[x])
        train_questions.append(questions[x])
        train_answers.append(answers[x])
    print(len(train_answers))
    ### 295 to end of doc
    for x in range(train_end,len(contexts)):
        val_contexts.append(contexts[x])
        val_questions.append(questions[x])
        val_answers.append(answers[x])
        
    print(len(val_answers))   
    return train_contexts, train_questions, train_answers,val_contexts, val_questions, val_answers

In [9]:
import urllib.request
import urllib, json
# url = "https://cb8e71e8ddfa.ngrok.io/ipc.json"
# response = urllib.request.urlopen(url)
# data = json.loads(response.read())
f = open('/content/drive/MyDrive/cdqa_ipc.json')
data = json.load(f)

In [10]:
print(str(data)[:100])

{'data': [{'title': 'S. 1 Title and extent of operation of the Code', 'paragraphs': [{'context': 'Ti


In [13]:

train_contexts, train_questions, train_answers,val_contexts, val_questions, val_answers = read_squad(data,0.8)

294
74


In [ ]:
print((train_contexts[0]))
print((train_questions[0]))
print((train_answers[0]))

Title and extent of operation of the Code
what is section 1 of IPC
{'answer_start': 0, 'text': 'Title and extent of operation of the Code'}


In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
# val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)


In [ ]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    print(len(answers))
    for i in range(len(answers)):
        try:
            start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
            end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
            # if None, the answer passage has been truncated
            if start_positions[-1] is None:
                start_positions[-1] = tokenizer.model_max_length
            if end_positions[-1] is None:
                end_positions[-1] = tokenizer.model_max_length
        except:
            print(answers[i],i)
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

294
74


In [ ]:
import torch

class IpcDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = IpcDataset(train_encodings)
val_dataset = IpcDataset(val_encodings)

In [ ]:
############# CONSIDER FROM HERE 

In [ ]:
from transformers import DistilBertForQuestionAnswering
model_db = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
'''this cell is code for fine tuning, we can ignore this'''

# from torch.utils.data import DataLoader
# from transformers import AdamW

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model_db.to(device)
# model_db.train()

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# optim = AdamW(model_db.parameters(), lr=5e-5)

# for epoch in range(10):
#     print(epoch)
#     for x,batch in enumerate(train_loader):
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         start_positions = batch['start_positions'].to(device)
#         end_positions = batch['end_positions'].to(device)
#         outputs = model_db(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()
#         if x == 10:
#             print("bi: "+str(x)+" loss: "+str(loss))
# model_db.eval()

In [ ]:
question="what is the punishment for preparing to commit dacoity?" 
# dated  22.7.2011,  passed  by  the  High Court of Judicature at Patna in Criminal Misc. No. 13116  of  2009  quashing  the criminal proceedings against the respondent  no. 2  while  allowing  the application under Section 482 of the Code of Criminal  Procedure,1973"
# "No.of quashing the criminal proceedings against the respondent no.while allowing the application under Section of the Code of Criminal Procedure, hereinafter referred to as Cr.P.C..,   Facts and circumstances giving rise to this petition are that: A."
# val_questions[2]
paragraph="Whoever makes any preparation for committing dacoity, shall be punished with rigorous imprisonment for a term which may extend to ten years, and shall also be liable to fine."
# val_contexts[2]
ans=""
# val_answers[2]
print(question, paragraph, ans)

what is the punishment for preparing to commit dacoity? Whoever makes any preparation for committing dacoity, shall be punished with rigorous imprisonment for a term which may extend to ten years, and shall also be liable to fine. 


In [ ]:
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)
input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
tokens = tokenizer.convert_ids_to_tokens(input_ids) #input tokens
print(tokens)
# print(input_ids, attention_mask)

['[CLS]', 'what', 'is', 'the', 'punishment', 'for', 'preparing', 'to', 'commit', 'da', '##co', '##ity', '?', '[SEP]', 'whoever', 'makes', 'any', 'preparation', 'for', 'committing', 'da', '##co', '##ity', ',', 'shall', 'be', 'punished', 'with', 'rigorous', 'imprisonment', 'for', 'a', 'term', 'which', 'may', 'extend', 'to', 'ten', 'years', ',', 'and', 'shall', 'also', 'be', 'liable', 'to', 'fine', '.', '[SEP]']


In [ ]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [ ]:
ip=torch.tensor([input_ids]).to(device)
attention=torch.tensor([attention_mask]).to(device)


In [ ]:
start_scores, end_scores = model_db(ip, attention)


In [ ]:
max_startscore = torch.argmax(start_scores)
max_endscore = torch.argmax(end_scores)
ans_tokens = input_ids[max_startscore: max_endscore + 1]
# answer = ' '.join(tokens[start_index:end_index+1])
print(ans_tokens)
answer = ' '.join(tokens[max_startscore:max_endscore+1])
print("question: "+str(question))
print("answer: "+str(answer))
print("context: "+str(paragraph))


In [ ]:




!pip install transformers==3.5.1
!pip install torch==1.6.0

     |████████████████████████████████| 1.3MB 4.1MB/s 
     |████████████████████████████████| 2.9MB 37.5MB/s 
     |████████████████████████████████| 1.1MB 41.4MB/s 
     |████████████████████████████████| 890kB 35.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1601e049d0439febf7b1232bf174012ae1fba9e469becc8b1e24438bd98ea7df
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 748.8MB 17kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [ ]:
from transformers import LongformerTokenizer, LongformerForQuestionAnswering
long_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")
long_model = LongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")

In [ ]:
context="Over the years, the global food system has brought nutrition,economic opportunity, convenience, and enjoyment. Today,however, the global impacts of climate change, soil erosion, waterscarcity, and population growth bring challenges that threaten itsability to meet our needs. The food system is in urgent need oftransformation. 70% of the world’s freshwater use is inagriculture alone PepsiCo’s sustainability agenda focuses on six overlappingpriorities within our food system. Our priorities meetthree important criteria: They relate to the most pressingsustainability challenges, risks, and opportunities facingPepsiCo and our food system; they matter most to PepsiCo’skey external stakeholders; and they offer the opportunityfor PepsiCo to make a positive difference at a systemic level,within and beyond our own value chain. At PepsiCo, we believe that there is an opportunity to change how the world produces,distributes, consumes, and disposes of foods and beverages in order to tackle the sharedchallenges we face. We aim to use our scale, reach, and expertise to help build a more sustainable food system; onethat can meet human needs for nutrition and enjoyment, and continue to drive economic andsocial development, without exceeding the natural boundaries of the planet. As well as being good for our business, working to realize this purpose is also the right thingto do. Where our food system may be operating in an unsustainable way, it can create supplychain, operational, and financial risks for our company. By contrast, addressing risks within ourvalue chain and the wider food system effectively can create new opportunities for competitiveadvantage and future market growth, while also supporting the communities in whichwe operate. HighlightsVideoLearn more about theprogress PepsiCo ismaking to help builda more sustainablefood system.View the video. In 2006, PepsiCo started on a journey to transform the way we do business so that we can deliver strong financial returns in a way that is responsive to the needs of theworld around us. Our fundamental belief is that the success of our company is inextricably linked to the sustainability of the world around us, and each year we continueto make valuable progress. PepsiColaunches“Performancewith Purpose”integratingsustainabilityinto dailybusinessoperations. PepsiCo’s firstall-electric deliverytrucks hit the road. PepsiCo launchesSustainable FarmingProgram, whichnow engages morethan 40,000 farmersaround the world. PepsiCo’s first DemonstrationFarm opens in India. In 2020, over230 farms around the world aretesting and sharing best practices. PepsiCo winsprestigious ClimateLeadership Award. PepsiCo prices thecompany’s first-everGreen Bond, proceedsfrom the U.S. $1 billionoffering will fund keysustainability initiatives. Naked Juice announcesplans to becomethe first nationallydistributed brand totransition to a 100%rPET bottle in the U.S. Launch of Food forGood, an employee-ledprogram that makesnutritious foods moreaccessible to low-incomechildren. PepsiCo winsprestigiousStockholmIndustryWater Award. PepsiCoU.K. delivers“50 in 5” —  50% reductionin carbon andwater in 5 years. PepsiCoachieves 25%operationalwater-useefficiencygoal ahead oftarget date. PepsiCopilotsindustrialcompostablesnack bags inChile, India,and the U.S. PepsiCoacquiresSodaStream,#1 sparklingwater brandin the worldby volume. PepsiCo launches 2025Sustainability Agendawith refreshed goals,designed to meetchanging consumer andsocietal needs. Launch of “All inRecycling,” the largestever industry-wideresidential recyclingchallenge, whichraises over $25 million inmatching funds and catalyzes$100 million in investment. PepsiCoannouncesshift to 100%renewableelectricity inits U.S. directoperations,building onglobal progress. PepsiCosigns theUN GlobalCompactBusinessAmbition for1.5°C pledge. PepsiCo exceeds safewater access goal5 years ahead ofschedule, deliveringsafe water access to44 million people in underservedcommunities since 2006 and"

In [ ]:
context1=context[:1000]

In [ ]:
question="Sustainability strategy of PepsiCo?"
import torch
encoding = long_tokenizer.encode_plus(question, context1)
input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

start_scores, end_scores = long_model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))

ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
answer_tokens = long_tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)

print ("\nQuestion ",question)
print ("\nAnswer Tokens: ")
print (answer_tokens)

answer_tokens_to_string = long_tokenizer.convert_tokens_to_string(answer_tokens)

print ("\nAnswer : ",answer_tokens_to_string)



Question  Sustainability strategy of PepsiCo?

Answer Tokens: 
['Ġfocuses', 'Ġon', 'Ġsix', 'Ġoverlapping', 'pri', 'or', 'ities', 'Ġwithin', 'Ġour', 'Ġfood', 'Ġsystem', '.', 'ĠOur', 'Ġpriorities', 'Ġmeet', 'three', 'Ġimportant', 'Ġcriteria', ':', 'ĠThey', 'Ġrelate', 'Ġto', 'Ġthe', 'Ġmost', 'Ġpress', 'ings', 'ustain', 'ability', 'Ġchallenges', ',', 'Ġrisks', ',', 'Ġand', 'Ġopportunities', 'Ġfacing', 'P', 'ep', 'si', 'Co', 'Ġand', 'Ġour', 'Ġfood', 'Ġsystem', ';', 'Ġthey', 'Ġmatter', 'Ġmost', 'Ġto', 'ĠPepsi', 'Co', 'âĢ', 'Ļ', 's', 'key', 'Ġexternal', 'Ġstakeholders', ';', 'Ġand', 'Ġthey', 'Ġoffer', 'Ġthe', 'Ġopportunity', 'for', 'ĠPepsi', 'Co', 'Ġto', 'Ġmake', 'Ġa', 'Ġpositive', 'Ġdifference', 'Ġat', 'Ġa', 'Ġsystemic', 'Ġlevel', ',', 'within', 'Ġand', 'Ġbeyond', 'Ġour', 'Ġown', 'Ġvalue', 'Ġchain', '.', 'ĠAt', 'ĠPepsi', 'Co']

Answer :   focuses on six overlappingpriorities within our food system. Our priorities meetthree important criteria: They relate to the most pressingsustainability c

In [ ]:
!pip install transformers==2.8.0
!pip install torch==1.4.0

     |████████████████████████████████| 573kB 17.4MB/s 
     |████████████████████████████████| 133kB 54.2MB/s 
     |████████████████████████████████| 1.2MB 55.2MB/s 
     |████████████████████████████████| 890kB 55.4MB/s 
     |████████████████████████████████| 3.7MB 53.6MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
     |████████████████████████████████| 7.2MB 53.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d4493cf4e6fca682fd38db9836bba9037e42f6f3efe53de3ad657e82423762bb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.57 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 753.4MB 22kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.4.0 which is incompatible.
 

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',return_token_type_ids = True)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')


In [ ]:
!pip install bert-extractive-summarizer


In [ ]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
file1 = open('/content/pepsi.txt', 'r') 
lines = file1.readlines() 

para=[]
headlines=[]
headline_latest=0
curr_para=""

for l in lines:
  if l.startswith(" ['"):
    headlines.append(l)
    headline_latest=1
  else:
    curr_para+=l
    if headline_latest==1:
      para.append(curr_para)
      curr_para=""
      headline_latest=0

##### remove \n 
para=[x.replace("\n","") for x in para]
para=[x.replace("\\n","") for x in para]
headlines=[x.replace("\n","") for x in headlines]
headlines=[x.replace("\\n","") for x in headlines]

updated_paras=[]
updated_headlines=[]
curr_head=headlines[0]
curr_para=""
for i, val in enumerate(headlines):

  if curr_head!=val:
    updated_paras.append(curr_para)
    updated_headlines.append(curr_head)
    curr_head=val
    curr_para=""
    curr_para+=" "+para[i]
  else:
    curr_para+=" "+para[i]

  


In [ ]:
summary_dict={}
for i, body in enumerate(updated_paras[:10]):
  result = model(body, ratio=0.2)
  print(i)
  summary_dict[result]=body

TypeError: ignored

In [ ]:
context=".".join(list(summary_dict.keys()))

In [ ]:

print ("\n\nContext : ",context)

questions = ["What are the challenges of the food system?",
             "What are Pepsico's focus areas on sustainability?"]
question_context_for_batch = []

for question in questions :
    question_context_for_batch.append((question, context))
encoding = tokenizer.batch_encode_plus(question_context_for_batch,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
start_scores, end_scores = model(input_ids, attention_mask=attention_mask)

for index,(start_score,end_score,input_id) in enumerate(zip(start_scores,end_scores,input_ids)):
    max_startscore = torch.argmax(start_score)
    max_endscore = torch.argmax(end_score)
    ans_tokens = input_ids[index][max_startscore: max_endscore + 1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens, skip_special_tokens=True)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    print ("\nQuestion: ",questions[index])
    print ("Answer: ", answer_tokens_to_string)



Context :  Over the years, the global food system has brought nutrition,economic opportunity, convenience, and enjoyment. The food system is in urgent need oftransformation..PepsiCo’s sustainability agenda focuses on six overlappingpriorities within our food system..At PepsiCo, we believe that there is an opportunity to change how the world produces,distributes, consumes, and disposes of foods and beverages in order to tackle the sharedchallenges we face..HighlightsVideoLearn more about theprogress PepsiCo ismaking to help builda more sustainablefood system..In 2006, PepsiCo started on a journey to transform the way we do business so that we can deliver strong financial returns in a way that is responsive to the needs of theworld around us..Naked Juice announcesplans to becomethe first nationallydistributed brand totransition to a 100%rPET bottle in the U.S. Launch of Food forGood, an employee-ledprogram that makesnutritious foods moreaccessible to low-incomechildren. PepsiCopilotsin

In [ ]:
context

'Over the years, the global food system has brought nutrition,economic opportunity, convenience, and enjoyment. The food system is in urgent need oftransformation..PepsiCo’s sustainability agenda focuses on six overlappingpriorities within our food system..At PepsiCo, we believe that there is an opportunity to change how the world produces,distributes, consumes, and disposes of foods and beverages in order to tackle the sharedchallenges we face..HighlightsVideoLearn more about theprogress PepsiCo ismaking to help builda more sustainablefood system..In 2006, PepsiCo started on a journey to transform the way we do business so that we can deliver strong financial returns in a way that is responsive to the needs of theworld around us..Naked Juice announcesplans to becomethe first nationallydistributed brand totransition to a 100%rPET bottle in the U.S. Launch of Food forGood, an employee-ledprogram that makesnutritious foods moreaccessible to low-incomechildren. PepsiCopilotsindustrialcomp

In [ ]:
summary_dict

{}

In [ ]:
updated_paras[0]

' Over the years, the global food system has brought nutrition,economic opportunity, convenience, and enjoyment. Today,however, the global impacts of climate change, soil erosion, waterscarcity, and population growth bring challenges that threaten itsability to meet our needs. The food system is in urgent need oftransformation. 70% of the world’s freshwater use is inagriculture alone'